In [79]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend


# CLS assignment 2: Networked part

In [80]:
# set MPL to render plots by chosing backend
mpl.use('TkAgg')

In [81]:
# assuming an undirected graph

# parameters

N = 1000 # number of nodes
p = 0.1 # probability link creation
# Example: Creating a simple graph with 1000 nodes and a 0.1 probability of edge creation
g = nx.erdos_renyi_graph(N, p)



In [82]:
SIR_model_randomgraph = ep.SIRModel(g)

In [83]:
config = mc.Configuration()
config.add_model_parameter('beta', 0.002)  # Infection probability
config.add_model_parameter('gamma', 0.05)  # Recovery probability
SIR_model_randomgraph.set_initial_status(config)

# Example: Infecting node 0
config.add_model_initial_configuration("Infected", 0.05*N)




C:\Users\nitai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ndlib\models\DiffusionModel.py:120: UserWarning: Initial infection missing: a random sample of 5% of graph nodes will be set as infected
  warnings.warn('Initial infection missing: a random sample of 5% of graph nodes will be set as infected')


In [84]:
iterations = SIR_model_randomgraph.iteration_bunch(200)  # Execute 200 iterations
trends = SIR_model_randomgraph.build_trends(iterations)

In [85]:
# for iteration, status in enumerate(iterations):
#     print(f"Iteration {iteration}:")
#     print("Node status:", status['status'])
#     print("-----")

In [86]:
len(trends)

1

In [87]:
trends[0]['trends']['node_count'][0][21]

483

In [88]:
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend

# Visualize the diffusion trend
viz = DiffusionTrend(SIR_model_randomgraph, trends)
viz.plot()
plt.show()


In [89]:
def simulate_sir_random_graph(N, k, network_type='erdos_renyi', p=None, beta=0.002, gamma=0.05, initial_i_proportion=0.1, iterations=200, graph=False):
    """
    Simulate an SIR model on Network based on an Erdos_Renyi model (default), Watts_Strogatz model or Barabasi_Albert model.

    Parameters:
    N (int): Total number of nodes in the graph.
    k (int): Expected degree of nodes in the graph.
    p (float, optional): Probability of edge creation between nodes. 
        Defaults to None, in which case it is calculated as k/(N-1).
    beta (float, optional): Infection probability per edge. Defaults to 0.002.
    gamma (float, optional): Recovery probability. Defaults to 0.05.
    initial_i_proportion (float, optional): Initial proportion of infected nodes. 
        Defaults to 0.1.
    iterations (int, optional): Number of iterations for the simulation. Defaults to 200.
    graph (bool, optional): Whether to plot the SIR trend or not. Defaults to False.

    Returns:
    tuple: A tuple containing the SIR model and the trends of the simulation.
        - SIR_model_randomgraph (ep.SIRModel): The SIR model on the random graph.
        - trends (list): A list of dictionaries containing the S, I, R counts per iteration.
    """
    
    # If p is not provided, calculate it based on k and N
    if p:
        p = p
    else:
        p = k/(N-1)
        
    # Creating a random graph using Erdős–Rényi model
    if network_type == 'erdos_renyi':
        network_type = nx.erdos_renyi_graph(N, p)
    elif network_type == 'watts_strogatz':
        network_type = nx.watts_strogatz_graph(N, k, p)
    elif network_type == 'barabasi_albert':
        network_type = nx.barabasi_albert_graph(N, k)

    sir_model= ep.SIRModel(network_type)
    
    # Setting up the SIR model with specified parameters
    initial_i = int(initial_i_proportion * N)
    config = mc.Configuration()
    config.add_model_parameter('beta', beta)  # Infection probability per edge
    config.add_model_parameter('gamma', gamma)  # Recovery probability
    sir_model.set_initial_status(config)
    config.add_model_initial_configuration("Infected", initial_i)

    # Running the SIR model for specified iterations
    iterations = sir_model.iteration_bunch(iterations) 
    trends = sir_model.build_trends(iterations)

    # Graphing the SIR trends if graph parameter is True
    if graph:
        viz = DiffusionTrend(sir_model, trends)
        viz.plot()

    return sir_model, trends


In [90]:
def get_network_statistics(graph):
    return {
        "Average Degree": sum(dict(graph.degree()).values()) / len(graph),
        "Clustering Coefficient": nx.average_clustering(graph),
        "Average Shortest Path Length": nx.average_shortest_path_length(graph)
    }


In [91]:
def simulate_sir_on_network(N, k, network_type='erdos_renyi', p=None, beta=0.002, gamma=0.05, initial_i_proportion=0.1, iterations=200, graph=False):
    """
    Simulate an SIR model on Network based on an Erdos_Renyi model (default), Watts_Strogatz model or Barabasi_Albert model.

    Parameters:
    N (float or list of float): Total number of nodes in the graph.
    k (float or list of float): Expected degree of nodes in the graph.
    p (float or list of float, optional): Probability of edge creation between nodes. 
        Defaults to None, in which case it is calculated as k/(N-1).
    beta (float, optional): Infection probability per edge. Defaults to 0.002.
    gamma (float, optional): Recovery probability. Defaults to 0.05.
    initial_i_proportion (float, optional): Initial proportion of infected nodes. 
        Defaults to 0.1.
    iterations (int, optional): Number of iterations for the simulation. Defaults to 200.
    graph (bool, optional): Whether to plot the SIR trend or not. Defaults to False.

    Returns:
    list of tuples: Each tuple contains the SIR model and the trends of the simulation for each combination of N, k, and p.
        - SIR_model_randomgraph (ep.SIRModel): The SIR model on the random graph.
        - trends (list): A list of dictionaries containing the S, I, R counts per iteration.
    """
    
    # Ensure N, k, and p are lists
    if not isinstance(N, list): N = [N]
    if not isinstance(k, list): k = [k]
    if not isinstance(p, list): p = [p]
    
    results = []
    
    for n in N:
        for degree in k:
            for probability in p:
                
                # If probability is not provided, calculate it based on degree and n
                if probability:
                    probability = probability
                else:
                    probability = degree/(n-1)

                # Creating a random graph
                if network_type == 'erdos_renyi':
                    network = nx.erdos_renyi_graph(int(n), probability)
                elif network_type == 'watts_strogatz':
                    network = nx.watts_strogatz_graph(int(n), int(degree), probability)
                elif network_type == 'barabasi_albert':
                    network = nx.barabasi_albert_graph(int(n), int(degree))

                sir_model= ep.SIRModel(network)

                # Setting up the SIR model with specified parameters
                initial_i = int(initial_i_proportion * n)
                config = mc.Configuration()
                config.add_model_parameter('beta', beta)  # Infection probability per edge
                config.add_model_parameter('gamma', gamma)  # Recovery probability
                sir_model.set_initial_status(config)
                config.add_model_initial_configuration("Infected", initial_i)

                # Running the SIR model for specified iterations
                iters = sir_model.iteration_bunch(iterations) 
                trends = sir_model.build_trends(iters)

                # Graphing the SIR trends if graph parameter is True
                if graph:
                    # Assuming DiffusionTrend is imported from ndlib.viz.mpl.DiffusionTrend
                    viz = DiffusionTrend(sir_model, trends)
                    viz.plot()
                    
                results.append((sir_model, trends))
                
    return results

In [92]:
# setting up simulation for multiple parameters

# parameters
N = 1000 # number of nodes
k_list = [0.5, 1, 5, 10] # expected degree of nodes in the graph
n_runs = 1 # number of runs per parameter

# running simulation for multiple parameters

In [93]:
beta_list = [0.002, 0.004, 0.01, 0.03, 0.05]
gamma_list = [0.05, 0.1, 0.15, 0.2, 0.25]
reverse_gamma_list = gamma_list[::-1]

In [94]:
def combined_network_statistics(graph, plot=False, caption=None):
    """
    Compute basic statistics, centrality measures, and optional visualizations for a given network/graph.

    Parameters:
    - graph (NetworkX graph): The graph for which statistics and visualizations are to be computed.
    - plot (bool, optional): If True, plots degree distribution, degree centrality, eigenvector centrality,
                             and betweenness centrality. Default is False.
    - caption (str, optional): Caption to be added to the bottom of the figure if plotting is enabled.
                               Default is None.

    Returns:
    - stats_df (pandas DataFrame): DataFrame containing basic statistics and centrality measures of the graph.
    - degree_dist_df (pandas DataFrame): DataFrame containing degree distribution of the graph.

    Notes:
    - Basic statistics include: Average Degree, Clustering Coefficient, Average Shortest Path Length, and Max Path Length.
    - Centrality measures include: Average Degree Centrality, Average Eigenvector Centrality, and Average Betweenness Centrality.
    - Plots are displayed in the order: Degree Distribution, Degree Centrality, Eigenvector Centrality, and Betweenness Centrality.
    - Ensure required libraries (e.g., NetworkX, matplotlib, pandas) are imported before using this function.
    """
    # Basic statistics
    stats = {
        "Average Degree": sum(dict(graph.degree()).values()) / len(graph),
        "Clustering Coefficient": nx.average_clustering(graph),
        "Average Shortest Path Length": nx.average_shortest_path_length(graph),
        "Max Path Length": nx.diameter(graph)
    }
    
    # Centrality measures
    degree_centrality = nx.degree_centrality(graph)
    eigenvector_centrality = nx.eigenvector_centrality(graph, max_iter=500)
    betweenness_centrality = nx.betweenness_centrality(graph)
    
    stats["Average Degree Centrality"] = sum(degree_centrality.values()) / len(graph)
    stats["Average Eigenvector Centrality"] = sum(eigenvector_centrality.values()) / len(graph)
    stats["Average Betweenness Centrality"] = sum(betweenness_centrality.values()) / len(graph)
    
    # Plotting
    if plot:
        plt.figure(figsize=(20, 5))
        
        # Degree Distribution
        degree_sequence = sorted([d for n, d in graph.degree()], reverse=True)
        plt.subplot(1, 4, 1)
        plt.hist(degree_sequence, bins=50)
        plt.title("Degree Distribution")
        plt.xlabel("Degree")
        plt.ylabel("Number of Nodes")
        
        # Degree Centrality
        plt.subplot(1, 4, 2)
        plt.hist(list(degree_centrality.values()), bins=50, color='skyblue')
        plt.title("Degree Centrality")
        plt.xlabel("Degree Centrality")
        plt.ylabel("Number of Nodes")
    
        # Eigenvector Centrality
        plt.subplot(1, 4, 3)
        plt.hist(list(eigenvector_centrality.values()), bins=50, color='salmon')
        plt.title("Eigenvector Centrality")
        plt.xlabel("Eigenvector Centrality")
        plt.ylabel("Number of Nodes")
    
        # Betweenness Centrality
        plt.subplot(1, 4, 4)
        plt.hist(list(betweenness_centrality.values()), bins=50, color='lightgreen')
        plt.title("Betweenness Centrality")
        plt.xlabel("Betweenness Centrality")
        plt.ylabel("Number of Nodes")
        
        # Add figure caption
        plt.figtext(0.5, -0.05, caption, ha="center", fontsize=12, bbox={"facecolor":"white", "alpha":0.5, "pad":5})
        
        plt.tight_layout()
        plt.show()
    
    # Convert statistics dictionary to DataFrame for better representation
    stats_df = pd.DataFrame([stats])
    degree_dist_df = pd.DataFrame({"Degree": degree_sequence})
    
    return stats_df, degree_dist_df

In [98]:
for beta,gamma in zip(beta_list[-3:-1], reverse_gamma_list[-3:-1]):
    # for k in k_list:
        for run in range(n_runs):
            sir_model_randomgraph, trends = simulate_sir_on_network(network_type='watts_strogatz', N=N, k=k, beta=beta, gamma=gamma, graph=True)[0]
            # combined_network_statistics(sir_model_randomgraph, plot=True)
            

In [ ]:
results = simulate_sir_on_network(N=1000, k=6, network_type='watts_strogatz', beta=0.002, gamma=0.05, graph=False)
sir_model = results[0][0]
network = sir_model.graph
stats_df, degree_df = combined_network_statistics(network, plot=True)



C:\Users\nitai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ndlib\models\DiffusionModel.py:120: UserWarning: Initial infection missing: a random sample of 5% of graph nodes will be set as infected
  warnings.warn('Initial infection missing: a random sample of 5% of graph nodes will be set as infected')


AttributeError: 'AGraph' object has no attribute 'degree'